In [1]:
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

from django.db.models import Sum, Max, Subquery, Q, F, Avg, Count, Min, Max, Sum, F, Q
from django.db.models import Value, IntegerField
from django.db.models.functions import Coalesce, Greatest, Floor
from account.models import User
from booth.models import Transaction, Participation
from player.models import Player

In [16]:
import random
random_index = {
    'money': [10000,15000,20000,25000,30000,35000,40000,45000,50000],
    'health_score': [20, 40, 60, 80, 100, 120],
    'skill_score': [20, 40, 60, 80, 100, 120, 130],
    'growth_score': [5, 10, 15, 20, 25, 30, 35],
    'relationship_score': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
    'academic_level': [1, 2, 3, 4]
}
admin = User.objects.get(id=999)
Player.objects.create(
    user=admin,
    born_money=random.choice(random_index['money']),
    born_health_score=random.choice(random_index['health_score']),
    born_skill_score=random.choice(random_index['skill_score']),
    born_growth_score=random.choice(random_index['growth_score']),
    born_relationship_score=random.choice(random_index['relationship_score']),
    born_academic_level=random.choice(random_index['academic_level']),
    born_steps=15
)

<Player: 999  >

In [18]:
admin.encrypted_id

'dBHubFozQwAJz5KVtTgvY78g1a0w0rbc'